<a href="https://colab.research.google.com/github/spberry4/KaggleTimeSeriesComp/blob/main/BooksSold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install darts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 5.1 MB/s 
     |████████████████████████████████| 9.8 MB 43.8 MB/s 
     |████████████████████████████████| 569 kB 27.9 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 76.8 MB 13 kB/s 
     |████████████████████████████████| 708 kB 38.0 MB/s 
     |████████████████████████████████| 77 kB 5.3 MB/s 
     |████████████████████████████████| 1.8 MB 11.1 MB/s 
     |████████████████████████████████| 11.2 MB 31.2 MB/s 
     |████████████████████████████████| 1.6 MB 34.7 MB/s 
     |████████████████████████████████| 960 kB 35.7 MB/s 
     |████████████████████████████████| 5.9 MB 30.3 MB/s 
     |████████████████████████████████| 529 kB 35.8 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstal

In [ ]:
import pandas as pd
from darts import TimeSeries
from darts.models import RegressionModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Data/train.csv', index_col="date", infer_datetime_format=True)

In [ ]:
train.tail()

,row_id,country,store,product,num_sold
date,,,,,
2020-12-31,70123,Spain,KaggleMart,Kaggle for Kids: One Smart Goose,614
2020-12-31,70124,Spain,KaggleRama,Kaggle Advanced Techniques,215
2020-12-31,70125,Spain,KaggleRama,Kaggle Getting Started,158
2020-12-31,70126,Spain,KaggleRama,Kaggle Recipe Book,135
2020-12-31,70127,Spain,KaggleRama,Kaggle for Kids: One Smart Goose,202


In [ ]:
train.index = pd.to_datetime(train.index)
train["num_sold"] = pd.to_numeric(train["num_sold"])

In [ ]:
train.index

DatetimeIndex(['2017-01-01', '2017-01-01', '2017-01-01', '2017-01-01',
               '2017-01-01', '2017-01-01', '2017-01-01', '2017-01-01',
               '2017-01-01', '2017-01-01',
               ...
               '2020-12-31', '2020-12-31', '2020-12-31', '2020-12-31',
               '2020-12-31', '2020-12-31', '2020-12-31', '2020-12-31',
               '2020-12-31', '2020-12-31'],
              dtype='datetime64[ns]', name='date', length=70128, freq=None)

In [ ]:
train.dtypes

row_id       int64
country     object
store       object
product     object
num_sold     int64
dtype: object

In [ ]:
print(train.shape)

(70128, 5)


In [ ]:
px.line(train, x = train.index, y = "num_sold", color = "product", title = "Books Sold")

In [ ]:
train.nunique()

row_id      70128
country         6
store           2
product         4
num_sold      699
dtype: int64

In [ ]:
train.isnull().sum()

row_id      0
country     0
store       0
product     0
num_sold    0
dtype: int64

In [ ]:
train["product"].unique()

array(['Kaggle Advanced Techniques', 'Kaggle Getting Started',
       'Kaggle Recipe Book', 'Kaggle for Kids: One Smart Goose'],
      dtype=object)

In [ ]:
train[train["product"] == 'Kaggle Advanced Techniques']

,row_id,country,store,product,num_sold
date,,,,,
2017-01-01,0,Belgium,KaggleMart,Kaggle Advanced Techniques,663
2017-01-01,4,Belgium,KaggleRama,Kaggle Advanced Techniques,240
2017-01-01,8,France,KaggleMart,Kaggle Advanced Techniques,610
2017-01-01,12,France,KaggleRama,Kaggle Advanced Techniques,220
2017-01-01,16,Germany,KaggleMart,Kaggle Advanced Techniques,700
...,...,...,...,...,...
2020-12-31,70108,Italy,KaggleRama,Kaggle Advanced Techniques,217
2020-12-31,70112,Poland,KaggleMart,Kaggle Advanced Techniques,606
2020-12-31,70116,Poland,KaggleRama,Kaggle Advanced Techniques,219


In [ ]:
#performing a decomposition and cleaning on all the products based on the unique products available
def clean_decomp_books(df, model):
  books = df["product"].unique()
  for word in books:
    #cleaning and prepping the data for time series decomposition analysis
    df_new = df[df["product"] == word]
    df_new = df_new.groupby(df_new.index).sum()
    df_new = df_new[["num_sold"]]
    df_new.asfreq('d')

    #Using the statmodel seasonal decomposition function to break apart the time series
    decompose_data = seasonal_decompose(df_new, model = model)
    
    #plotting each of the books decompositions
    fig = make_subplots(rows=2, cols=2, subplot_titles=("Observed", "Trend", "Seasonal", "Residuals"))

    fig.add_trace(
    go.Scatter(x = decompose_data.observed.index, y = decompose_data.observed, name = "Observed"),
    row=1, col=1)

    fig.add_trace(
    go.Scatter(x = decompose_data.trend.index, y = decompose_data.trend, name = "Trend"),
    row=1, col=2)

    fig.add_trace(
    go.Scatter(x = decompose_data.seasonal.index, y = decompose_data.seasonal, name = "Seasonal"),
    row=2, col=1)

    fig.add_trace(
    go.Scatter(x = decompose_data.resid.index, y = decompose_data.resid, name = "Residuals"),
    row=2, col=2)

    fig.update_layout(hovermode="x unified",
                 showlegend=False,
                 title = f"{word} Time Series Decomposition")


    fig.show()

In [ ]:
clean_decomp_books(train, "additive")

In [ ]:
def feature_extraction(df):
  books = df["product"].unique()
  for word in books:
    df_new = df[df["product"] == word]
    df_new = df_new.groupby(df_new.index).sum()
    #df_new = df_new[["num_sold"]]
    df_new.asfreq('d')

    df_new["month"] = pd.DatetimeIndex(df_new["date"]).month
    df_new["year"] = pd.DatetimeIndex(df_new["date"]).year

    

In [ ]:
train.head()

,date,country,store,product,num_sold,month,year
row_id,,,,,,,
0,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,663,1,2017
1,2017-01-01,Belgium,KaggleMart,Kaggle Getting Started,615,1,2017
2,2017-01-01,Belgium,KaggleMart,Kaggle Recipe Book,480,1,2017
3,2017-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,710,1,2017
4,2017-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,240,1,2017


In [ ]:
y_all = TimeSeries.from_group_dataframe(train,
                                        group_cols=["country", "store", "product"],
                                        time_col="date",
                                        value_cols="num_sold")

In [ ]:
future_cov_all = TimeSeries.from_group_dataframe(train,
                                        group_cols=["country", "store", "product"],
                                        time_col="date",
                                        value_cols=["month", "year"])

In [ ]:
model = RegressionModel(lags = [-1,-8, -12],
                        lags_future_covariates=[0],
                        model = LinearRegression())

In [ ]:
model.fit(y_all, future_covariates=future_cov_all)

In [ ]:
model.predict(n=12,
              series = y_all,
              future_covariates = future_cov_all)

ERROR:main_logger:ValueError: The corresponding future_covariate of the series at index 0 isn't sufficiently long. Given horizon `n=12`, `min(lags_future_covariates)=0`, `max(lags_future_covariates)=0` and `output_chunk_length=1`
the future_covariate has to range from 2021-01-01 00:00:00 until 2021-01-12 00:00:00 (inclusive), but it ranges only from 2017-01-01 00:00:00 until 2020-12-31 00:00:00.


ValueError: ignored